In [1]:
# %%
%load_ext autoreload
%autoreload 2

In [4]:
# %% imports
import pandas as pd
import numpy as np
from scripts.signals import Signal, MeanReversion, CrossSectionCorrelation
from scripts.filters import VolumeFilter, FixedVolumeFilter
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
# %% loading data
raw_data = pd.read_csv('../data/data_f2009.csv', parse_dates=['date']).set_index(['date', 'ticker'])
data = raw_data.sort_index()

In [441]:
datasample = data.loc[:"2013-12-17"]

In [442]:
datasample.head()

adj_volume  adj_close  future_perf_1d  past_perf_1d
date       ticker                                                      
2009-01-05 A       5129700.000  11.454821        0.079809      0.033867
           AAL     3457100.145  21.644190        0.087635     -0.007151
           AAN        2500.500  14.233812        0.050228      0.025761
           AAON    1258399.125   5.626286       -0.001433      0.002395
           AAP      969700.000  32.795583        0.001485     -0.013474

In [443]:
# Because WAG is now WBA
datasample.drop("WAG", axis=0, level=1, inplace=True)

/Users/marc/.miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [444]:
# %% filter data
nb_stocks = 500
datasample['filter'] = FixedVolumeFilter(datasample, nb_stocks, "2009-01-05").get_filter()

/Users/marc/.miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/marc/.miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [445]:
kept_tickers = sorted(datasample["filter"].loc(axis=0)[datasample["filter"] == 1].index.get_level_values(1).unique())

In [ ]:
# TODO invert matrices once/refit. 

In [446]:
# %% create signal 
# ---- 
# note that we need more n_past than number of kept stocks if we want
# the correlation matric to be non-singular. 
# We need to put lots of n_past days in order for GARCH models to converge. 
mr_signal = CrossSectionCorrelation(datasample, n_past=200, n_fit=3, refit=30)
datasample['signal'] = mr_signal.get_signal()

2009-10-23 00:00:00
Starting new series fit.
Minimum eigenvalue of R is (-8.425977206177682e-15+0j)


/Users/marc/Documents/Dropbox/Stanford/Class/3rdQuarter/MS&E448/algotrading/scripts/signals/garch.py:128: RuntimeWarning: invalid value encountered in sqrt
  sigma1 = np.sqrt(self.initial_sigma[index, 0] - np.dot(rho, np.dot(H_inv, rho.T))[0, 0])


2009-10-26 00:00:00
2009-10-27 00:00:00
2009-10-28 00:00:00
2009-10-29 00:00:00
2009-10-30 00:00:00
2009-11-02 00:00:00


KeyboardInterrupt: 

In [ ]:
datasample.signal[~datasample.signal.isna()].groupby(level=0).apply(lambda x: np.mean((x < 1e-8) | (x > 1 - 1e-8))).plot()

In [ ]:
from scipy.stats import norm
diffs = datasample.signal[~datasample.signal.isna()]
diffs[diffs == 0] = .5
diffs[diffs == 1] = .5
plt.hist(norm.ppf(diffs), bins=100)
plt.yscale("log")
plt.show()

In [ ]:
# %% compute perfs
data_perf = datasample.loc[datasample['filter'] == 1].dropna(subset=['signal'])
data_perf['position'] = data_perf.signal.groupby(level=0).apply(lambda x: x - np.mean(x))
data_perf['perf'] = data_perf.position * data_perf.future_perf_1d
data_perf['to'] = data_perf.position.groupby(level=1).diff().fillna(0).abs()
data_perf['not'] = data_perf.position.fillna(0).abs()

# %% perfs
print('Sharpe: ' + "{0:.3f}".format(16 * data_perf.perf.groupby(
    level=0).sum().mean() / data_perf.perf.groupby(level=0).sum().std()))
print('rbt: ' + "{0:.3f}".format(100 * data_perf.perf.groupby(
    level=0).sum().sum() / data_perf.to.groupby(level=0).sum().sum()) + "%")
print('holding: ' + "{0:.2f}".format(2 * data_perf['not'].groupby(
    level=0).sum().sum() / data_perf.to.groupby(level=0).sum().sum()))
data_perf.perf.groupby('date').sum().cumsum().plot(figsize=(12, 7))

# data_computed.loc(axis=0)[:, 'HHS'].reset_index(level=1, drop=True).close.plot(figsize=(16, 7))